In [109]:
### 한글 폰트 설정
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
    
matplotlib.rcParams['axes.unicode_minus'] = False

%matplotlib inline

In [110]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [112]:
import pandas as pd

train = pd.read_csv("./train_df_errno.csv")
test = pd.read_csv("./test_df.csv")
sub = pd.read_csv("./sample_submission.csv")
age = pd.read_csv("./age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [114]:
all_df = pd.concat([train, test], join='inner')

### train, test 데이터 합쳐서 결측치 확인 : 자격유형, 임대보증금, 임대료, 10분내지하철수, 10분내버스정류장수

In [116]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
dtype: int64

### 자격유형 결측치 처리

In [119]:
all_df.loc[ 196, "자격유형"] = 'A'
all_df.loc[ 258, "자격유형"] = 'C'


In [135]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }

all_df['자격유형'] =all_df['자격유형'].map(mapping).astype(int)

In [136]:
all_df.loc[ train['10분내버스정류장수'].isnull(), :]


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,1,46000000,456000,NaN,4.0,1066.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,1,57000000,462000,NaN,4.0,1066.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,1,57000000,462000,NaN,4.0,1066.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,1,57000000,462000,NaN,4.0,1066.0


In [137]:
all_df['임대건물구분'].unique()

array(['아파트', '상가'], dtype=object)

In [138]:
grouped = train.groupby(['임대건물구분', '지역'])
group1 = grouped.get_group(('아파트','경상남도'))
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
5,C2515,545,아파트,경상남도,국민임대,51.97,106,17.0,A,23042000,190090,0.0,3.0,624.0,205.0
6,C2515,545,아파트,경상남도,국민임대,51.97,26,17.0,A,23042000,190090,0.0,3.0,624.0,205.0
84,C2576,405,아파트,경상남도,국민임대,46.90,313,19.0,A,19436000,161780,0.0,4.0,296.0,73.0
85,C2576,405,아파트,경상남도,국민임대,59.88,92,19.0,A,28252000,235420,0.0,4.0,296.0,73.0
123,C2416,560,아파트,경상남도,영구임대,26.34,360,0.0,C,3138000,69820,0.0,5.0,154.0,63.0


In [139]:
all_df.loc[all_df['10분내버스정류장수'].isnull(),'10분내버스정류장수'] = 5
all_df.loc[all_df['10분내버스정류장수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


In [140]:
train_df = all_df.iloc[0:2896,:]
test_df = all_df.iloc[2896:,:]

train_df.shape, test_df.shape

train_df = pd.concat([train_df, train['등록차량수'] ], axis=1)
train_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,205.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,1,11346000,116090,0.0,1.0,166.0,146.0
2892,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,1,14005000,142310,0.0,1.0,166.0,146.0
2893,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,1,14005000,142310,0.0,1.0,166.0,146.0
2894,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,1,14005000,142310,0.0,1.0,166.0,146.0


In [141]:
from sklearn.model_selection import train_test_split

In [142]:
train_df.corr()['등록차량수'].sort_values(ascending=False)


등록차량수         1.000000
단지내주차면수       0.861338
총세대수          0.333440
전용면적별세대수      0.250513
공가수           0.118910
전용면적          0.112717
10분내버스정류장수    0.104203
10분내지하철수     -0.107308
자격유형         -0.154034
Name: 등록차량수, dtype: float64

In [143]:
sel = [ '총세대수', '전용면적', '전용면적별세대수', '공가수', '자격유형', '10분내버스정류장수', 
       '단지내주차면수']

X = train_df[sel]
y = train_df['등록차량수']
test_X = test_df[sel]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    random_state=0)

In [144]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [145]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.7837160530373486
테스트(score) : 0.8125520891303324


In [146]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

135.34006905860738
135.34006905860738
188.21922889050026


In [147]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9987804193243294
테스트(score) : 0.9955625292639592


In [149]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mse_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

13.77389655172414
838.6541051724134
28.95952529259438


In [152]:
!pip install xgboost

In [153]:
import xgboost as xgb

In [154]:

data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
# 기본 옵션 확인
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 0.33, # 각나무마다 사용하는 feature 비율
            learning_rate = 0.1,
            max_depth = 5, 
            alpha = 0.1, 
            n_estimators = 2500)  # n_estimators=2500

xg_reg.fit(X_train, y_train)
pred = xg_reg.predict(X_test)

print("학습(score) :", xg_reg.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", xg_reg.score(X_test, y_test) ) # 결정계수

[01:46:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
학습(score) : 0.9999145435691145
테스트(score) : 0.9929640981565726


In [155]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

22.062679132921943
22.062679132921943
36.46561761849794


In [159]:
train.shape, test.columns

((2896, 15),
 Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수'],
       dtype='object'))

In [160]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(test_X)
pred[0:10]


array([ 617.3 ,  607.95,  612.93,  612.93,  616.71,  617.87,  623.96,
        621.46, 1229.64, 1231.53])

In [162]:
test_df['등록차량수'] = pred
test_df['단지별차량수평균'] = test_df.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test_df.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

<ipython-input-162-3cc158f3592c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['등록차량수'] = pred
<ipython-input-162-3cc158f3592c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['단지별차량수평균'] = test_df.groupby("단지코드")['등록차량수'].transform(np.mean)


,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,단지별차량수평균
0,0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0,617.30,616.388750
1,8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,8,22830000,189840,0.0,3.0,1216.0,1229.64,1239.194444
2,17,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,1,19706000,156200,0.0,16.0,547.0,536.03,555.323333
3,26,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,1,14418000,108130,0.0,3.0,543.0,508.45,552.927500
4,30,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,8,28598000,203050,0.0,2.0,1112.0,1004.56,997.273750
5,38,C1563,1974,아파트,경기도,국민임대,39.73,214,15.0,1,30040000,206380,0.0,6.0,1696.0,1379.82,1378.010000
6,49,C1794,1349,아파트,전라북도,국민임대,29.91,245,25.0,1,10273000,92330,0.0,2.0,1098.0,866.04,902.213333
7,55,C1640,533,아파트,전라북도,국민임대,39.51,87,17.0,1,15620000,114140,0.0,1.0,470.0,326.57,357.805000
8,59,C1377,470,아파트,경상남도,국민임대,29.91,62,18.0,1,8394000,71950,0.0,4.0,384.0,414.12,371.166000
9,64,C2072,353,아파트,경상남도,국민임대,33.61,116,6.0,1,8050000,68110,0.0,1.0,280.0,305.78,310.202500


In [163]:
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [164]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,616.389
1,C1128,1239.19
2,C1456,555.323
3,C1840,552.928
4,C1332,997.274
5,C1563,1378.01
6,C1794,902.213
7,C1640,357.805
8,C1377,371.166
9,C2072,310.202


In [166]:
sub_df.to_csv('fourth_rf_0719.csv', index=False)
sub_df.head()

,code,num
0,C1072,616.389
1,C1128,1239.19
2,C1456,555.323
3,C1840,552.928
4,C1332,997.274
